In [1]:
!pip install -r requirements.txt

  Using cached pyautogen-0.2.27-py3-none-any.whl.metadata (23 kB)
  Using cached anthropic-0.26.1-py3-none-any.whl.metadata (18 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached FLAML-2.1.2-py3-none-any.whl.metadata (15 kB)
  Using cached openai-1.30.3-py3-none-any.whl.metadata (21 kB)
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.6 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_

In [2]:
import sys
sys.path.append("lib/src/")
from autogen_utils import AnthropicClient

config_list_claude = [
    {
        "model": "anthropic.claude-3-sonnet-20240229-v1:0",
        "api_type": "anthropic",
        "model_client_cls": "AnthropicClient",
        "cache_seed" : None, #we deactivate the cache on purpose
    }
]

#see below the 2 main modifications that we have to include into the code to make that Anthropic wrapper work in the code

#llm_config={"config_list": config_list_claude}
#agent.register_model_client(model_client_cls=AnthropicClient)

# Introduction to AutoGen

Welcome! AutoGen is an open-source framework that leverages multiple _agents_ to enable complex workflows. This tutorial introduces basic concepts and building blocks of AutoGen.

## Why AutoGen?

> _The whole is greater than the sum of its parts._<br/>
> -**Aristotle**

While there are many definitions of agents, in AutoGen, an agent is an entity that can send messages, receive messages and generate a reply using models, tools, human inputs or a mixture of them.
This abstraction not only allows agents to model real-world and abstract entities, such as people and algorithms, but it also simplifies implementation of complex workflows as collaboration among agents.

Further, AutoGen is extensible and composable: you can extend a simple agent with customizable components and create workflows that can combine these agents and power a more sophisticated agent, resulting in implementations that are modular and easy to maintain.

Most importantly, AutoGen is developed by a vibrant community of researchers
and engineers. It incorporates the latest research in multi-agent systems
and has been used in many real-world applications, including agent platform,
advertising, AI employees, blog/article writing, blockchain, calculate burned areas by wildfires,
customer support, cybersecurity, data analytics, debate, education, finance, gaming, legal consultation,
research, robotics, sales/marketing, social simulation, software engineering,
software security, supply chain, t-shirt design, training data generation, Youtube service...

## Installation

The simplest way to install AutoGen is from pip: `pip install pyautogen`. Find more options in [Installation](/docs/installation/).

## Agents

In AutoGen, an agent is an entity that can send and receive messages to and from
other agents in its environment. An agent can be powered by models (such as a large language model
like GPT-4), code executors (such as an IPython kernel), human, or a combination of these
and other pluggable and customizable components.

```{=mdx}
![ConversableAgent](./assets/conversable-agent.jpg)
```

An example of such agents is the built-in `ConversableAgent` which supports the following components:

1. A list of LLMs
2. A code executor
3. A function and tool executor
4. A component for keeping human-in-the-loop

You can switch each component on or off and customize it to suit the need of 
your application. For advanced users, you can add additional components to the agent
by using [`registered_reply`](../reference/agentchat/conversable_agent/#register_reply). 

LLMs, for example, enable agents to converse in natural languages and transform between structured and unstructured text. 
The following example shows a `ConversableAgent` with a GPT-4 LLM switched on and other
components switched off:

In [3]:
import os

from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": config_list_claude},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

#added
agent.register_model_client(model_client_cls=AnthropicClient)

[autogen.oai.client: 05-27 07:06:57] {426} INFO - Detected custom model client in config: AnthropicClient, model client can not be used until register_model_client is called.


The `llm_config` argument contains a list of configurations for the LLMs.
See [LLM Configuration](/docs/topics/llm_configuration) for more details.

You can ask this agent to generate a response to a question using the `generate_reply` method:

In [4]:
reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)

Here's a silly joke for you:

Why did the tomato turn red? Because it saw the salad dressing!


## Roles and Conversations

In AutoGen, you can assign roles to agents and have them participate in conversations or chat with each other. A conversation is a sequence of messages exchanged between agents. You can then use these conversations to make progress on a task. For example, in the example below, we assign different roles to two agents by setting their
`system_message`.

In [5]:
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": config_list_claude},
    human_input_mode="NEVER",  # Never ask for human input.
)

[autogen.oai.client: 05-27 07:07:35] {426} INFO - Detected custom model client in config: AnthropicClient, model client can not be used until register_model_client is called.
[autogen.oai.client: 05-27 07:07:35] {426} INFO - Detected custom model client in config: AnthropicClient, model client can not be used until register_model_client is called.


In [7]:
#added
cathy.register_model_client(model_client_cls=AnthropicClient)
joe.register_model_client(model_client_cls=AnthropicClient)

[autogen.oai.client: 05-27 07:09:47] {465} WARNING - Model client AnthropicClient is already registered. Add more entries in the config_list to use multiple model clients.
[autogen.oai.client: 05-27 07:09:47] {465} WARNING - Model client AnthropicClient is already registered. Add more entries in the config_list to use multiple model clients.


Now that we have two comedian agents, we can ask them to start a comedy show.
This can be done using the `initiate_chat` method.
We set the `max_turns` to 2 to keep the conversation short.

In [8]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.", max_turns=2)

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

*straightens bowtie and clears throat* Okay, here's one for you! Why can't a bicycle stand up by itself? It's two-tired! *waits for laughter, drums on knee* Ah, come on, that was a classic! My comedy partner Pat is usually better at this banter thing than me. Want to hear another?

--------------------------------------------------------------------------------
joe (to cathy):

*chuckles* Not bad, not bad! Though I've heard that one wheely too many times before. But hey, I'm always game for more jokes from the comedic genius of Joe and Pat. Hit me with another zinger!

--------------------------------------------------------------------------------
cathy (to joe):

*slicks back hair and puffs out chest confidently* Alright, alright, you asked for it! Why do seagulls fly over the sea? Because if they flew over the bay, they would be bagels! *waits fo

The comedians are bouncing off each other!

## Summary

In this chapter, we introduced the concept of agents, roles and conversations in AutoGen.
For simplicity, we only used LLMs and created fully autonomous agents (`human_input_mode` was set to `NEVER`). 
In the next chapter, 
we will show how you can control when to _terminate_ a conversation between autonomous agents.